In [1]:
def set_gpu_memory_growth_mode(gpu_id=0):
    import tensorflow as tf
    try:
        gpus = tf.config.experimental.list_physical_devices('GPU')
        tf.config.experimental.set_memory_growth(gpus[gpu_id], True)
    except RuntimeError as e:
        print(e)

In [2]:
set_gpu_memory_growth_mode()

In [3]:
import sys
import math

import ds.data4 as ds

from cheat.tfs import *
from matrix import *
from context import *
from nets.m4 import *

np.set_printoptions(suppress=True)

# Context

In [4]:
config = {
    'seed' : 1234,
    'use_64bits': False,
    
    # Loss
    'loss_type': 'mse',
    
    # LEO & target parameter theta dim
    'num_latents': 16*4,
    # 'gen_theta_dim': 96*8 + 8,
    'gen_theta_dim': 3,
    'num_k_shot': 5,
    'num_valid_shot': 10,
        
    # Batch & Step size
    'batch_size' : 3,
    'first_decay_steps': 20,
    'meta_lr': 1e-4,
    'latent_lr': 1e-6,
    'finetuning_lr': 1e-6,
    'num_latent_grad_steps' : 5,
    'num_finetune_grad_steps' : 5,
    'num_meta_grad_steps' : 5,
    
    'gradient_threshold': 0.2,
    'gradient_norm_threshold': 0.2,
 
    # Regularizer Term
    'dropout_rate': 0.4,
    'kl_weight': 1e-3,
    'l2_penalty_weight': 1e-8,
    'encoder_penalty_weight': 1e-9,
    'orthogonality_penalty_weight': 1e-3,
    
    'num_epochs' : 20,
    'npy_root_path': '/home/elvin/banner/mnt/ssd3/nps/',
}

ctx = Context.create(config)

In [5]:
data_provider = ds.DataProvider(ctx, "01054")

In [6]:
# ASSERT 
num = len(data_provider)
tasks = data_provider[num-5]
task = tasks[ctx.batch_size-1]

assert task.tr_le.shape == (ctx.num_k_shot, 64, 128, 1)
assert task.tr_re.shape == (ctx.num_k_shot, 64, 128, 1)
assert task.tr_output.shape == (ctx.num_k_shot, 5)
assert task.val_output.shape == (ctx.num_valid_shot, 5)

In [7]:
mat = Matrix(ctx)
net = Leo.create(ctx)

In [8]:
test_task_ids = np.random.choice(len(data_provider), 6)
test_task_ids = np.random.choice(len(data_provider), 6)
print(test_task_ids)

[24 57 15 49 23 41]


In [9]:
# for i in range(8):
#    net.run_with_test(50, data_provider, mat, test_task_ids, prior_task_id=0)

In [10]:
vars_zero = net.trainable_variables
print([v.name for v in vars_zero])

['latent_lr:0', 'finetuning_lr:0']


In [11]:
vars_e_zero = net.eyenet.trainable_variables
print([v.name for v in vars_e_zero])

[]


In [ ]:
net.run_with_test(1, data_provider, mat, test_task_ids, prior_task_id=0)

# Layer

In [13]:
vars = net.trainable_variables
[(i,v.name) for i,v in enumerate(vars)]

[(0, 'encoder_l1/kernel:0'),
 (1, 'encoder_l2/kernel:0'),
 (2, 'relation_l1/kernel:0'),
 (3, 'relation_l2/kernel:0'),
 (4, 'relation_l3/kernel:0'),
 (5, 'decoder/kernel:0'),
 (6, 'fc_eye/kernel:0'),
 (7, 'fc_gaze_vec/kernel:0'),
 (8, 'fc_gaze_vec/bias:0'),
 (9, 'fc_target/kernel:0'),
 (10, 'latent_lr:0'),
 (11, 'finetuning_lr:0')]

In [14]:
vars[8]

<tf.Variable 'fc_gaze_vec/bias:0' shape=(3,) dtype=float32, numpy=array([-0.00193635,  0.00030106, -0.00102999], dtype=float32)>

In [17]:
evars = net.eyenet.trainable_variables
[(i,ev.name) for i,ev in enumerate(evars)]

[(0, 'dl_bn1/gamma:0'),
 (1, 'dl_bn1/beta:0'),
 (2, 'dl_bn2/gamma:0'),
 (3, 'dl_bn2/beta:0'),
 (4, 'dl_bn3/gamma:0'),
 (5, 'dl_bn3/beta:0'),
 (6, 'dl_conv1/kernel:0'),
 (7, 'dl_conv2/kernel:0'),
 (8, 'dl_conv3/kernel:0'),
 (9, 'dw_conv1/depthwise_kernel:0'),
 (10, 'dw_conv_l1/depthwise_kernel:0'),
 (11, 'dw_conv_r1/depthwise_kernel:0'),
 (12, 'fc1/kernel:0'),
 (13, 'in_norm1/gamma:0'),
 (14, 'in_norm1/beta:0'),
 (15, 'in_norm2/gamma:0'),
 (16, 'in_norm2/beta:0'),
 (17, 'in_norm3/gamma:0'),
 (18, 'in_norm3/beta:0'),
 (19, 'in_norm_f1/gamma:0'),
 (20, 'in_norm_f1/beta:0'),
 (21, 'in_norm_l1/gamma:0'),
 (22, 'in_norm_l1/beta:0'),
 (23, 'in_norm_r1/gamma:0'),
 (24, 'in_norm_r1/beta:0'),
 (25, 'in_norm_r2/gamma:0'),
 (26, 'in_norm_r2/beta:0'),
 (27, 'pw_conv1/kernel:0'),
 (28, 'pw_conv2/kernel:0'),
 (29, 'pw_conv_l1/kernel:0'),
 (30, 'pw_conv_r1/kernel:0'),
 (31, 'pw_conv_r2/kernel:0'),
 (32, 'stem_bn1/gamma:0'),
 (33, 'stem_bn1/beta:0'),
 (34, 'stem_conv1/kernel:0')]

In [19]:
evars[12].numpy()

array([[-0.01321553,  0.06336878, -0.18351857, ..., -0.07619655,
         0.07227682,  0.04707609],
       [-0.04093336, -0.18346262, -0.02021189, ...,  0.11297508,
        -0.16489351,  0.07567738],
       [ 0.19219302, -0.0417639 , -0.00523867, ...,  0.18024515,
        -0.17390998, -0.15407884],
       ...,
       [ 0.04827158,  0.1819617 ,  0.11742558, ..., -0.13749918,
         0.0090666 , -0.09392995],
       [ 0.15983643, -0.09682325,  0.04287954, ...,  0.20372333,
        -0.19453466, -0.14520562],
       [ 0.191346  ,  0.16680686, -0.21235374, ...,  0.04997487,
         0.00828564, -0.2117336 ]], dtype=float32)